# Prepare database

In [25]:
import os

dir_name = 'C:/Users/windo/Documents/ravdess-dataset/audio_speech_actors_01-24'

# Initialize a list to hold the file names
all_files = []

# Traverse all subfolders and files using os.walk()
for root, _, file_names in os.walk(dir_name):
    for file_name in file_names:
        file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):  # Check if it's a file
            all_files.append(file_path)

all_files = sorted(all_files)

# Display the collected file names
# print('List of all files in the folders:')
# for file in all_files:
#     print(file)

# Emotions in the RAVDESS dataset
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# desired csv columns:
# label/name(actor number, take, etc) emotion(class) feature1 feature2...
# or class at the end?
len(all_files)

1440

In [26]:
import numpy as np

import sys
sys.path.append("..")
import plap

# Use plap for parameterization
fvector = plap.FeatureVector("asc","ass","asf","lat","tc","sc","hsc","hsd","hss","hsv")
preprocessor = plap.Preprocessor(window_type="hamming")

def extract_features(audio_path):
    plap.parameterize(audio_path=audio_path, fvector=fvector, preprocessor=preprocessor)
    return fvector.values

In [27]:
import csv

# Define the CSV filename
csv_filename = './python_csv/ravdess_basspectr_timbral.csv'

# Initialize the CSV file with headers
headers = ['label', 'class'] + [f'{feature.upper()}' for feature in fvector.features]
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)

In [ ]:
# Iterate over each audio file
for filename in all_files:
    if filename.endswith('.wav'):  # Ensure only audio files are processed
        
        # Extract label and class from the filename (customize this logic as needed)
        audio_filename = filename.split('\\')[-1]
        label = audio_filename.removesuffix('.wav')  # Use filename as label
        class_num = audio_filename.split('-')[2]
        class_name = emotions[class_num]
        
        print(f'Processing file {audio_filename}')

        # Extract 11 features from the audio file
        features = extract_features(filename)
        # features = np.zeros(11)

        # Prepare the row to be added to the CSV
        row = [label, class_name] + list(features)

        # Append the new row to the CSV file
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(row)

print("Feature extraction and CSV writing completed.")

Processing file 03-01-04-01-01-01-11.wav
Processing file 03-01-06-02-01-01-12.wav
Processing file 03-01-01-01-01-01-14.wav
Processing file 03-01-04-01-01-01-15.wav
Processing file 03-01-06-02-01-01-16.wav
Processing file 03-01-01-01-01-01-18.wav
Processing file 03-01-04-01-01-01-19.wav
Processing file 03-01-06-02-01-01-20.wav
Processing file 03-01-01-01-01-01-22.wav
Processing file 03-01-04-01-01-01-23.wav
Feature extraction and CSV writing completed.
